In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os


from google.colab import drive
drive.mount('/content/drive')# Custom F1Score metric (make sure this matches the one used in training)
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Set up constants
IMG_SIZE = (224, 224)  # Adjust if your models use a different input size
BATCH_SIZE = 32

# Define the path to the images folder
data_dir = '/content/drive/My Drive/images'  # Update this path if necessary

# Define class names
class_names = ['MEL', 'NV', 'BCC', 'SCC']

def evaluate_model(model_path):
    # Load the model
    model = load_model(model_path, custom_objects={'F1Score': F1Score})

    # Set up data generator for evaluation
    eval_datagen = ImageDataGenerator(rescale=1./255)

    eval_generator = eval_datagen.flow_from_directory(
        data_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=class_names,
        shuffle=False
    )

    # Evaluate the model
    scores = model.evaluate(eval_generator, verbose=1)

    # Print the results
    print("\nEvaluation results:")
    for metric, score in zip(model.metrics_names, scores):
        print(f"{metric}: {score}")

    # Evaluate on specific classes
    for class_name in class_names:
        class_generator = eval_datagen.flow_from_directory(
            data_dir,
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            classes=[class_name],
            shuffle=False
        )
        scores = model.evaluate(class_generator, verbose=0)
        print(f"\nEvaluation on {class_name} class:")
        for metric, score in zip(model.metrics_names, scores):
            print(f"{metric}: {score}")

# Main loop
while True:
    model_path = input("Enter the path to the model file (or 'q' to quit): ")

    if model_path.lower() == 'q':
        break

    if not os.path.exists(model_path):
        print(f"Error: File not found at {model_path}")
        continue

    try:
        evaluate_model(model_path)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

print("Evaluation complete. Thank you!")

Mounted at /content/drive
Enter the path to the model file (or 'q' to quit): from google.colab import drive drive.mount('/content/drive')
Error: File not found at from google.colab import drive drive.mount('/content/drive')
Enter the path to the model file (or 'q' to quit): /content/drive/My Drive/models/skin_lesion_classifier_resnet50_final.keras
Found 8985 images belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


281/281 ━━━━━━━━━━━━━━━━━━━━ 5174s 18s/step - accuracy: 0.9193 - f1_score: 0.9192 - loss: 0.2455 - precision: 0.9195 - recall: 0.9189

Evaluation results:
loss: 0.37821459770202637
compile_metrics: 0.8888146877288818
Found 1113 images belonging to 1 classes.
An error occurred: Graph execution error:

Detected at node LogicalAnd defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [1,32] vs. [1,128]
	 [[{{node LogicalAnd}}]]
	tf2xla conversion failed while converting __inference_one_step_on_data_23081[]. Run with TF_DUMP_GRAPH_PREFIX=/path/to/dump/dir and --vmodule=xla_compiler=2 to obtain a dump of the compiled functions.
	 [[StatefulPartitionedCall]] [Op:__inference_one_step_on_iterator_23764]
Enter the path to the model file (or 'q' to quit): q
Evaluation complete. Thank you!
